In [137]:
import pandas as pd
import datetime as dt
import yfinance as yf
import json
import os
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# get watch_list

In [138]:
with open('watch_list.json') as f:
    watch_list = json.load(f)

In [139]:
watch_list
watch_list=['ACE',
 'ADVANC',
 'AEONTS']
watch_list

['ACE', 'ADVANC', 'AEONTS']

# fetch data from yfinance api

In [140]:
# create stock_data directory if it doesn't exist
if not os.path.exists('stock_data'):
    os.makedirs('stock_data')

# loop through watch_list and download data for each stock
for symbol in watch_list:
    tk = yf.Ticker(f"{symbol}.BK")
    tk.history(period='2y',interval='1d').to_csv(f"stock_data/{symbol}.csv")

# plot data with plotly 

##### (https://python.plainenglish.io/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682)

In [141]:
symbol = watch_list[0]

In [142]:

##############    read data into pandas dataframe   ##############
df = pd.read_csv(f"stock_data/{symbol}.csv")
df.set_index('Date', inplace=True)

In [143]:

##############        apply ta to dataframe         ##############
df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)
df.ta.ema(close='Close', length=200, append=True)

df = df.tail(100)
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_200
Date,,,,,,,,,,,
2022-07-08,2.72,2.72,2.66,2.68,10563400,0.0,0,-0.032958,-0.000808,-0.032150,3.177052
2022-07-11,2.68,2.76,2.66,2.72,31432700,0.0,0,-0.031752,0.000319,-0.032070,3.172505
2022-07-12,2.70,2.72,2.66,2.68,6063700,0.0,0,-0.033635,-0.001252,-0.032383,3.167604
2022-07-14,2.68,2.70,2.64,2.66,6479100,0.0,0,-0.036323,-0.003152,-0.033171,3.162553
2022-07-15,2.66,2.66,2.62,2.66,6881500,0.0,0,-0.038015,-0.003875,-0.034140,3.157553


In [150]:

##############       plot data using plotly         ##############
fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.01,
                    row_heights=[0.7, 0.3])
####  create figure with ohlc data  ####
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             hoverlabel=dict(bgcolor='white'),
                             name=symbol
                             ),
              row=1, col=1)


fig.update_layout(title=symbol)

####       remove range slider      ####
fig.update_layout(xaxis_rangeslider_visible=False)

####    removing all empty dates    ####

# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0], end=df.index[-1])

# retrieve the dates that ARE in the original dataset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]

# define dates with missing values
dt_breaks = [d for d in dt_all.strftime(
    "%Y-%m-%d").tolist() if not d in dt_obs]

# remove dates that are missing from the original dataset
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])


####    add EMA_200     ####
fig.add_trace(go.Scatter(x=df.index,
                         y=df['EMA_200'],
                         opacity=0.7,
                         line=dict(color='purple', width=2),
                         name='EMA_200'))

####     add MACD       ####
colors = ['green' if val >= 0 
          else 'red' for val in df['MACDh_12_26_9']]

fig.add_trace(go.Bar(x=df.index,
                     y=df['MACDh_12_26_9'],
                     name='MACDh_12_26_9',
                     marker_color=colors
                     ), row=2, col=1,)
fig.add_trace(go.Scatter(x=df.index,
                         y=df['MACD_12_26_9'],
                         line=dict(color='black', width=2),
                         name='MACD_12_26_9'
                         ), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=df['MACDs_12_26_9'],
                         line=dict(color='blue', width=1),
                         name='MACDs_12_26_9'
                         ), row=2, col=1)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)
fig.update_layout(margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=40  #top margin
    ))

fig.write_image("fig.png", width=1200, height=800, scale=4)
fig


In [154]:
df = pd.DataFrame()
df = df.ta.ticker("ACE.BK", period="2y", interval="1d")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-15,3.914832,3.993920,3.875289,3.895061,34993500,0.0,0
2020-07-16,3.914832,3.934604,3.835745,3.855517,16303300,0.0,0
2020-07-17,3.855517,3.895061,3.776429,3.835745,20341300,0.0,0
2020-07-20,3.835745,3.875289,3.776429,3.776429,15678300,0.0,0
2020-07-21,3.815973,3.875289,3.796201,3.855517,15607100,0.0,0
...,...,...,...,...,...,...,...
2022-07-08,2.720000,2.720000,2.660000,2.680000,10563400,0.0,0
2022-07-11,2.680000,2.760000,2.660000,2.720000,31432700,0.0,0
2022-07-12,2.700000,2.720000,2.660000,2.680000,6063700,0.0,0
